In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.externals import joblib
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#from PostStemmer import PostStemmer
#from sklearn.naive_bayes import MultinomialNB
#from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
#from sklearn.pipeline import Pipeline
#from sklearn.base import TransformerMixin
#from sklearn.feature_selection import chi2, SelectKBest
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.cross_validation import train_test_split, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [3]:
#first_post = pd.read_json('data/politicos/json/mixed.json')
reddit_pos = pd.read_json('data/politicos/json/multi.json')
reddit_neg = pd.read_json('data/politicos/json/single.json')
first_post = pd.concat([reddit_pos, reddit_neg], axis=0)

In [4]:
#first_post.reset_index(inplace=True)
#first_post.head()

In [5]:
# Author- not needed in analysis, keep for now as index
first_post.set_index('author', inplace=True)
#first_post.drop('author', axis=1, inplace=True)

In [6]:
# Label / outcome variable
multi_post = first_post['total_posts']>1
first_post.drop(['total_posts', 'post_ids', 'post_datetimes', 'last_post_datetime'], axis=1, inplace=True)
multi_post.index.rename('multi_post', inplace=True)
multi_post.name = 'multi_post'

In [7]:
# Datetimes
pdt = pd.to_datetime(first_post['first_post_datetime'])
first_post.drop('first_post_datetime', axis=1, inplace=True)

# Date == days since Jan 1 2007
delta = pdt - pd.Timestamp('01-01-2007')
first_post['date'] = delta / np.timedelta64(1, 'D')

# Days of week (baseline Monday)
first_post['tues']    = pdt.apply(lambda ts: 1 if ts.dayofweek==1 else 0)
first_post['wed']     = pdt.apply(lambda ts: 1 if ts.dayofweek==2 else 0)
first_post['thurs']   = pdt.apply(lambda ts: 1 if ts.dayofweek==3 else 0)
first_post['fri']     = pdt.apply(lambda ts: 1 if ts.dayofweek==4 else 0)
first_post['sat']     = pdt.apply(lambda ts: 1 if ts.dayofweek==5 else 0)
first_post['sun']     = pdt.apply(lambda ts: 1 if ts.dayofweek==6 else 0)
#first_post['weekend'] = pdt.apply(lambda ts: 1 if ts.dayofweek==5 or ts.dayofweek==6 else 0)

# Daily and yearly cycles
first_post['time_of_day'] = pdt.apply(lambda ts: ts.minute + 60 * ts.hour)
first_post['day_of_year'] = pdt.apply(lambda ts: ts.dayofyear)

In [8]:
# Ups/downs
first_post.rename(columns={'first_post_ups':'ups',
                           'first_post_downs':'downs'}, inplace=True)
first_post['has_ups'] = first_post['ups'].apply(lambda ups: 1 if ups > 0 else 0)
# no downs in this sample

In [9]:
# Sentiment
sentiment_clf = joblib.load('twitter_sentiment/classifier.pkl')

In [10]:
# Response sentiment

response_sentiment = [] #proportion of responses that were positive
#sentiments = []
#pos_responses = []
#neg_responses = []

for i,responses in enumerate(first_post['first_post_responses']):
    if isinstance(responses, float): #NaNs due to no responses
        #sentiments.append(None)
        #pos_responses.append(0)
        #neg_responses.append(0)
        response_sentiment.append(0)
    else: #utf8 encoding for classifier;
        responses_utf8 = [response.encode('utf8') for response in responses]
        responses_utf8_np = np.array(responses_utf8, ndmin=1)
        responses_utf8_np.reshape(responses_utf8_np.shape[0],1) #explicit 1-dimension for classifier
        responses_sentiments = sentiment_clf.predict(responses_utf8_np)
        #sentiments.append(responses_sentiments)
        #pos_responses.append(np.sum(responses_sentiments))
        #neg_responses.append(responses_sentiments.shape[0] - np.sum(responses_sentiments))
        response_sentiment.append( np.sum(responses_sentiments) / responses_sentiments.shape[0] )

first_post['responses_sentiment'] = response_sentiment
#first_post['pos_responses'] = pos_responses
#first_post['neg_responses'] = neg_responses

In [11]:
# Body sentiment
first_post_utf8 = first_post['first_post_body'].apply(lambda post: post.encode('utf8'))
first_post['sentiment'] = sentiment_clf.predict(first_post_utf8).astype(int)

In [12]:
# Responses
first_post['responses_avg_word_ct'] = first_post['first_post_responses'].apply(lambda responses: 0 if isinstance(responses, float) else np.sum([len(response.split()) for response in responses]) * 1.0 / len(responses) )
first_post.rename(columns={'first_post_avg_response_ups':'responses_ups_avg',
                           'first_post_avg_response_downs':'responses_downs_avg',
                           'first_post_total_responses':'responses_total'}, inplace=True)
first_post.drop(['first_post_responses','first_post_response_ups', 'first_post_response_downs'], axis=1, inplace=True) #not doing text analysis for now
first_post.fillna(0, inplace=True) #response stats are NaN if no responses
# responses_ups_avg is actually an interaction term multiplied by has_responses
first_post['has_responses'] = (first_post['responses_total']>0).astype(int)

In [13]:
# Body
first_post['word_count'] = first_post['first_post_body'].apply(lambda post: len(post.split()))
first_post.drop(['first_post_link_id', 'first_post_id', 'first_post_body'], axis=1, inplace=True)

In [14]:
# Parent type
first_post['is_response'] = first_post['parent_type']=='t1'
first_post['is_response'] = first_post['is_response'].astype(int)
first_post.drop('parent_type', axis=1, inplace=True)

In [15]:
# Downs and Response downs - drop bc no data in this sample
first_post.drop('responses_downs_avg', axis=1, inplace=True)
first_post.drop('downs', axis=1, inplace=True)

In [16]:
# Const
first_post = sm.add_constant(first_post)
#first_post['const'] = pd.Series(np.ones(first_post.shape[0]))

In [17]:
first_post.head()

,const,responses_ups_avg,responses_total,ups,date,tues,wed,thurs,fri,sat,...,weekend,time_of_day,day_of_year,has_ups,responses_sentiment,sentiment,responses_avg_word_ct,has_responses,word_count,is_response
multi_post,,,,,,,,,,,,,,,,,,,,,
Loreat,1,0,0,1,2123.712963,0,1,0,0,0,...,0,1026,298,1,0,0,0,0,46,0
alc0,1,0,0,2,2052.127523,1,0,0,0,0,...,0,183,227,1,0,1,0,0,13,1
tyofson,1,0,0,1,2119.889931,0,0,0,0,1,...,1,1281,294,1,0,0,0,0,10,0
Climaximis,1,0,0,5,2465.641123,1,0,0,0,0,...,0,923,274,1,0,0,0,0,16,0
virmundi,1,2,1,0,1347.658113,0,0,1,0,0,...,0,947,252,0,0,0,148,1,81,1


In [32]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score

rfc = RandomForestClassifier(n_estimators=10, max_features=3, max_depth=5)

print 'accuracy', cross_val_score(rfc, first_post, multi_post, scoring='accuracy')
print 'precision', cross_val_score(rfc, first_post, multi_post, scoring='precision')
print 'recall', cross_val_score(rfc, first_post, multi_post, scoring='recall')

accuracy [ 0.61094225  0.61094225  0.62068966]
precision [ 0.63181818  0.62745098  0.62378378]
recall [ 0.95695364  0.9205298   0.93366501]


In [33]:
rfc.fit(first_post, multi_post)
feat_impt = zip(first_post.columns, rfc.feature_importances_)
sorted(feat_impt, key=lambda (colname, score): score, reverse=True)

[('date', 0.19850003838149283),
 ('word_count', 0.13947398578983411),
 ('has_responses', 0.089742082308330834),
 ('day_of_year', 0.085063838357629623),
 ('responses_avg_word_ct', 0.082385990075867194),
 ('ups', 0.082342064508728635),
 ('time_of_day', 0.071466010393125445),
 ('responses_total', 0.064271538782350507),
 ('is_response', 0.041687624825666188),
 ('responses_ups_avg', 0.040529674022543254),
 ('sat', 0.034929555826398097),
 ('wed', 0.015281767627147025),
 ('has_ups', 0.015008839507544041),
 ('sun', 0.014069653937769915),
 ('sentiment', 0.0077106226529192511),
 ('tues', 0.0054561662752249922),
 ('responses_sentiment', 0.0050629877936641445),
 ('fri', 0.0032906365821672888),
 ('weekend', 0.0024513200572205463),
 ('thurs', 0.0012756022943761201),
 ('const', 0.0)]

In [35]:
logit = sm.Logit(multi_post, first_post)
result = logit.fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.640985
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             multi_post   No. Observations:                 2960
Model:                          Logit   Df Residuals:                     2940
Method:                           MLE   Df Model:                           19
Date:                Sun, 16 Aug 2015   Pseudo R-squ.:                 0.04033
Time:                        23:12:03   Log-Likelihood:                -1897.3
converged:                       True   LL-Null:                       -1977.1
                                        LLR p-value:                 3.194e-24
=========================================================================================
                            coef    std err          z      P>|z|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------------
const                     1.1709      0.222      5.284      0.000         0.737     1.605
responses_ups_avg         0.0104      0.015      0.705      0.481        -0.019     0.039
responses_total           0.0068      0.045      0.152      0.879        -0.081     0.095
ups                       0.0004      0.003      0.141      0.888        -0.005     0.006
date                     -0.0005   7.21e-05     -7.546      0.000        -0.001    -0.000
tues                      0.0188      0.142      0.132      0.895        -0.259     0.297
wed                       0.0379      0.141      0.268      0.788        -0.239     0.315
thurs                    -0.0502      0.141     -0.355      0.722        -0.328     0.227
fri                      -0.0743      0.142     -0.523      0.601        -0.352     0.204
sat                      -0.3434      0.151     -2.269      0.023        -0.640    -0.047
sun                       0.0597      0.151      0.395      0.693        -0.237     0.356
time_of_day           -9.834e-05   8.78e-05     -1.120      0.263        -0.000  7.38e-05
day_of_year              -0.0001      0.000     -0.281      0.778        -0.001     0.001
has_ups                   0.0640      0.104      0.615      0.538        -0.140     0.268
responses_sentiment      -0.0215      0.172     -0.125      0.901        -0.358     0.315
sentiment                -0.0292      0.086     -0.340      0.734        -0.198     0.139
responses_avg_word_ct     0.0028      0.002      1.676      0.094        -0.000     0.006
has_responses             0.4411      0.132      3.337      0.001         0.182     0.700
word_count                0.0016      0.001      2.218      0.027         0.000     0.003
is_response               0.3818      0.081      4.734      0.000         0.224     0.540
=========================================================================================
"""

In [21]:
np.exp(result.params)

const                    3.224882
responses_ups_avg        1.010497
responses_total          1.006858
ups                      1.000397
date                     0.999456
tues                     1.018970
wed                      1.038669
thurs                    0.950995
fri                      0.928421
sat                      0.779701
sun                      1.166842
weekend                  0.909781
time_of_day              0.999902
day_of_year              0.999899
has_ups                  1.066121
responses_sentiment      0.978768
sentiment                0.971210
responses_avg_word_ct    1.002804
has_responses            1.554428
word_count               1.001576
is_response              1.464895
dtype: float64

In [22]:
result.aic

3834.6317072757106

In [84]:
logit2 = sm.Logit(multi_post, first_post[[  'date',
                                            'word_count', 
                                            'has_ups', 
                                            'has_responses', 
                                            'day_of_year', 
                                            #'time_of_day', 
                                            #'responses_ups_avg', 
                                            'is_response', 
                                            'responses_avg_word_ct']])
result2 = logit2.fit()
result2.summary()

Optimization terminated successfully.
         Current function value: 0.648517
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             multi_post   No. Observations:                 2960
Model:                          Logit   Df Residuals:                     2953
Method:                           MLE   Df Model:                            6
Date:                Sun, 16 Aug 2015   Pseudo R-squ.:                 0.02906
Time:                        23:47:48   Log-Likelihood:                -1919.6
converged:                       True   LL-Null:                       -1977.1
                                        LLR p-value:                 1.919e-22
=========================================================================================
                            coef    std err          z      P>|z|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------------
date                     -0.0002   4.64e-05     -5.166      0.000        -0.000    -0.000
word_count                0.0020      0.001      2.832      0.005         0.001     0.003
has_ups                   0.3446      0.088      3.897      0.000         0.171     0.518
has_responses             0.5414      0.096      5.612      0.000         0.352     0.730
day_of_year               0.0007      0.000      1.978      0.048      6.15e-06     0.001
is_response               0.3846      0.079      4.845      0.000         0.229     0.540
responses_avg_word_ct     0.0026      0.002      1.584      0.113        -0.001     0.006
=========================================================================================
"""

In [85]:
result2.aic

3853.2188066988238

In [86]:
# Convert word counts to 10-word counts 
result2_params = result2.params.copy()
result2_params['responses_avg_word_ct'] = result2_params['responses_avg_word_ct'] *10
result2_params['word_count'] = result2_params['word_count'] *10

In [87]:
odds_pvals = pd.concat([np.exp(result2_params), result2.pvalues], axis=1)
odds_pvals.columns = ['Odds Ratio', 'P-val']
odds_pvals.sort('Odds Ratio', ascending=False, inplace=True)
pd.options.display.float_format = '{:,.4f}'.format
odds_pvals
# note these are odds ratios for 10-word counts

,Odds Ratio,P-val
has_responses,1.7184,0.0000
is_response,1.4691,0.0000
has_ups,1.4114,0.0001
responses_avg_word_ct,1.0265,0.1133
word_count,1.0207,0.0046
day_of_year,1.0007,0.0479
date,0.9998,0.0000


In [ ]:
result2.pvalues

Getting feedback on first posts is associated with posting again in Reddit Politics.
- Receiving at least one response to first comment associated with 71% better odds of posting again (p &lt; 0.001)
- Receiving at least one upvote to first comment associated with 41% better odds of posting again (p &lt; 0.001)
    
Participating in dialog was also associated with posting again. Users whose first comment was in response to another comment had 47% better odds of posting again (p &lt; 0.001). (This was after controlling for similar number of responses and upvotes, so it's not because responding to others makes them more likely to get responses in return.)

Length of first comments and responses to first comments were somewhat associated with higher probability of posting again, although the effect size was smaller and statistical significance was weaker. For each additional 10 words in their first comment, users had 2.1% higher odds of posting again (p=0.005). For each additional 10 words on average in the responses to their first comment, users had 2.7% higher odds of posting again, although the result was not statistically significant (p=0.113).

All results were controlled for yearly seasonality, and absolute date across site history.


In [28]:
odds_ratios = np.exp(result2.params).sort(ascending=False, inplace=False)
print odds_ratios

has_responses            1.640937
is_response              1.512106
responses_ups_avg        1.009108
responses_avg_word_ct    1.002661
word_count               1.002130
ups                      1.001295
day_of_year              1.000861
time_of_day              1.000131
date                     0.999824
dtype: float64


In [ ]:
features = first_post[[ 'date',
                        'word_count', 
                        'ups', 
                        'has_responses', 
                        'day_of_year', 
                        'time_of_day', 
                        'responses_ups_avg', 
                        'is_response', 
                        'responses_avg_word_ct' ]]
sk_logit = LogisticRegression(fit_intercept=False)
accuracy_cv = cross_val_score(sk_logit, X=features, y=multi_post, scoring='accuracy', cv=5)
print 'model accuracy', np.mean(accuracy_cv)
auc_cv = cross_val_score(sk_logit, X=features, y=m
                         ulti_post, scoring='roc_auc', cv=5)
print 'model auc', np.mean(auc_cv)

In [29]:
np.exp(result3.params)

NameError: name 'result3' is not defined

Yahoo Answers study, which included first-week engagement:
```
model accuracy: 0.691
model auc: 0.758
```

In [ ]:
sk_dummy = DummyClassifier()
accuracy_cv = cross_val_score(sk_dummy, X=features, y=multi_post, scoring='accuracy', cv=5)
print 'model accuracy', np.mean(accuracy_cv)
auc_cv = cross_val_score(sk_dummy, X=features, y=multi_post, scoring='roc_auc', cv=5)
print 'model auc', np.mean(auc_cv)

Logistic regression model predicts ~20% more accurately than random guess.

In [ ]:
logit3 = sm.Logit(multi_post, first_post[[  'date',
                                            'word_count', 
                                            #'ups', 
                                            'has_responses', 
                                            'day_of_year', 
                                            'time_of_day', 
                                            #'responses_ups_avg', 
                                            'is_response', 
                                            'responses_avg_word_ct' ]])
result3 = logit3.fit()
result3.summary()

In [ ]:
result3.aic

In [ ]:
response_crosstab_scaled = response_crosstab / response_crosstab.sum(axis=0)
response_crosstab_scaled

In [ ]:
response_crosstab_scaled.T.plot(kind="bar", stacked=True)
plt.title('Proportion who commented again,\n depending on whether or not they \nreceived a response to their first comment')

In [ ]:
first_post.word_count.hist()

In [ ]:
plt.figure()
plt.hist(first_post[first_post['ups']>0]['ups'], bins=75)
plt.xlim((0,80))
plt.title('Upvote distribution')

In [ ]:
word_count_discrete = pd.cut(first_post.word_count, [0,50,100,150,200,800])

In [ ]:
word_count_crosstab = pd.crosstab(multi_post, word_count_discrete)
word_count_crosstab

In [ ]:
word_count_crosstab_scaled = word_count_crosstab / word_count_crosstab.sum(axis=0)
word_count_crosstab_scaled

In [ ]:
word_count_crosstab_scaled.T.plot(kind="bar", stacked=True)

More responses to mid-length comments -- consider feature for mid-length comments

In [ ]:
first_post.ups.hist()

In [ ]:
ups_discrete = pd.cut(first_post.ups, [-100,-0.5,0.5,10,100,1000])

In [ ]:
ups_crosstab = pd.crosstab(multi_post, ups_discrete)
ups_crosstab

In [ ]:
ups_crosstab_scaled = ups_crosstab / ups_crosstab.sum(axis=0)
ups_crosstab_scaled.T.plot(kind="bar", stacked=True)

Highly upvoted first_post more likely to repost

Are ups and has_responses collinear?

In [ ]:
ups_vs_has_responses = pd.crosstab(ups_discrete, first_post.has_responses)
ups_vs_has_responses.T

In [ ]:
ups_vs_has_responses_scaled = ups_vs_has_responses.T / ups_vs_has_responses.sum(axis=1)
ups_vs_has_responses_scaled

In [ ]:
ups_vs_has_responses_scaled.T.plot(kind="bar", stacked=True)

In [ ]:
first_post.apply(np.log).plot('ups','responses_total', kind="scatter")

More upvotes is correlated with having responses